# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [50]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [51]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)
# For loop to create a list of files and collect each filepath
file_path_list = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*events.csv'))
    for f in files:
        file_path_list.append(os.path.abspath(f))
# Printing The files paths contained in file_path_list
file_path_list.sort()
print(len(file_path_list))
print(file_path_list)

/home/workspace
/home/workspace/event_data
30
['/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [52]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called 'event_datafile_new' that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [53]:
# check the number of rows in 'event_datafile_new.csv'
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [54]:
# Load 'event_datafile_new.csv' into a pandas dataframe to show some details
df = pd.read_csv('event_datafile_new.csv')
df.head(n = 16)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8
5,Blue October / Imogen Heap,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",139,Congratulations,8
6,Girl Talk,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",139,Once again,8
7,Black Eyed Peas,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",9,Pump It,10
8,Fall Out Boy,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",169,Nobody Puts Baby In The Corner,26
9,M.I.A.,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",169,Mango Pickle Down River (With The Wilcannia Mob),26


In [55]:
# shape of data (number of rows and columns)
df.shape

(6820, 11)

In [56]:
# Try to simulate The first Query using pandas dataframe 
print(df[ ((df['sessionId']==338) & (df['itemInSession']==4))]) 

        artist firstName gender  itemInSession  lastName    length level  \
964  Faithless       Ava      F              4  Robinson  495.3073  free   

                  location  sessionId                             song  userId  
964  New Haven-Milford, CT        338  Music Matters (Mark Knight Dub)      50  


In [57]:
# Try to simulate The Second Query using pandas dataframe 
print(df[ ((df['userId']==10) & (df['sessionId']==182))]) 

                artist firstName gender  itemInSession lastName     length  \
151   Down To The Bone    Sylvie      F              0     Cruz  333.76608   
152       Three Drives    Sylvie      F              1     Cruz  411.63710   
153  Sebastien Tellier    Sylvie      F              2     Cruz  377.73016   
154      Lonnie Gordon    Sylvie      F              3     Cruz  181.21098   

    level                                      location  sessionId  \
151  free  Washington-Arlington-Alexandria, DC-VA-MD-WV        182   
152  free  Washington-Arlington-Alexandria, DC-VA-MD-WV        182   
153  free  Washington-Arlington-Alexandria, DC-VA-MD-WV        182   
154  free  Washington-Arlington-Alexandria, DC-VA-MD-WV        182   

                                                  song  userId  
151                                 Keep On Keepin' On      10  
152                                        Greece 2000      10  
153                                          Kilometer      10 

In [58]:
# Try to simulate The Third Query using pandas dataframe 
print(df[df['song']=='All Hands Against His Own'])

              artist   firstName gender  itemInSession lastName     length  \
219   The Black Keys        Sara      F             31  Johnson  196.91057   
2442  The Black Keys  Jacqueline      F             50    Lynch  196.91057   
2646  The Black Keys       Tegan      F             25   Levine  196.91057   

     level                           location  sessionId  \
219   paid                  Winston-Salem, NC        152   
2442  paid  Atlanta-Sandy Springs-Roswell, GA        559   
2646  paid        Portland-South Portland, ME        611   

                           song  userId  
219   All Hands Against His Own      95  
2442  All Hands Against His Own      29  
2646  All Hands Against His Own      80  


# Part II. Apache Cassandra : Loading the data into The corresponding Tables

## Next we work with The generated CSV file titled <font color=red>event_datafile_new.csv</font>.  
## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cassandra Cluster Instance and Session

In [59]:
from cassandra.cluster import Cluster
try:
    # Set up a cassandra cluster
    cluster = Cluster(['127.0.0.1'])
    # Establish connection to the cluster and create session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [60]:
CREATE_KEYSPACE_QUERY = """
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = {
            'class':'SimpleStrategy', 
            'replication_factor': 1
            }
        """

try:
    session.execute(CREATE_KEYSPACE_QUERY)
except Exception as e:
    print(e)

#### Set Keyspace

In [61]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### First Query : SELECT artist, length, song FROM music_library WHERE sessionid = 338 AND iteminsession = 4

#### ✱ Interpretation : 
- This query tries to get songs details such as name, artist and length during a specific session 

#### ✱ Expected Output :
- Artist's Name, length of the Song and Song Name

#### ✱ Primary Key :
- (sessionid, iteminsession) as a Composite Partition Key

#### ✱ Column names :
- artist, iteminsession, length, sessionid, song

#### In the next cell We Will create the music_library_by_sessions table corresponding to the First Query

In [62]:
create_table1 = """CREATE TABLE IF NOT EXISTS music_library_by_sessions (
        artist text, 
        iteminsession int, 
        length double, 
        sessionid int, 
        song text,
        
        PRIMARY KEY (sessionid, iteminsession)
        )"""
try:
    session.execute(create_table1)
except Exception as e:
    print(e)

#### In the next cell we parse the 'event_datafile_new.csv' file row by row and extract the specific columns data and load it into the First Table "music_library_by_sessions"

In [63]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO music_library_by_sessions (
                artist, 
                iteminsession, 
                length, 
                sessionid, 
                song
                ) 
        """
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9] ) ) 

#### Execute The First Query

In [64]:
query1 = """SELECT artist, length, song
            FROM music_library_by_sessions
            WHERE sessionid = 338 AND iteminsession = 4
"""
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Second Query : SELECT artist, song, firstname, lastname FROM music_library WHERE userid = 10 AND sessionid = 182

#### ✱ Interpretation : 
- This query tries to get songs details such as name and artist and User's details such Name during a specific session 

#### ✱ Expected Output :
- Artist's Name, Song Name , firstname and lastname of the User

#### ✱ Primary Key :
- (userid, sessionid) as a Composite Partition Key and iteminsession as a Clustering Column

#### ✱ Column names :
- artist, iteminsession, song, firstname, lastname, sessionid, userid

#### In the next cell We Will create the music_library_by_users_and_sessions table corresponding to the Second Query

In [65]:
create_table2 = """CREATE TABLE IF NOT EXISTS music_library_by_users_and_sessions (
        artist text,
        iteminsession int,
        song text,
        firstname text,
        lastname text,
        sessionid int,
        userid int, 
        
        PRIMARY KEY ((userid, sessionid), iteminsession) 
        )"""
try:
    session.execute(create_table2)
except Exception as e:
    print(e)



#### In the next cell we parse the 'event_datafile_new.csv' file row by row and extract the specific columns data and load it into the Second Table "music_library_by_users_and_sessions"

In [66]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO music_library_by_users_and_sessions (
                artist,
                iteminsession,
                song,
                firstname, 
                lastname,  
                sessionid,  
                userId) 
        """
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], int(line[3]), line[9], line[1], line[4], int(line[8]), int(line[10])))

#### Execute The Second Query

In [67]:
query2 = """SELECT artist, song, firstname, lastname
            FROM music_library_by_users_and_sessions
            WHERE userid = 10 AND sessionid = 182

"""
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Third Query : SELECT firstname, lastname FROM music_library WHERE song = 'All Hands Against His Own'

#### ✱ Interpretation : 
- This query tries to get all the users (name and last name) who have listened to a specific song

#### ✱ Expected Output :
- firstname and lastname of the User Who have listened to a song

#### ✱ Primary Key :
- (song, userid) Composite Partition Key to make Primary Keys Unique

#### ✱ Column names :
- firstname, lastname, song, userid

#### In the next cell We Will create the music_library_by_songs table corresponding to the Third Query

In [68]:
create_table3 = """CREATE TABLE IF NOT EXISTS music_library_by_songs (
        firstname text, 
        lastname text,  
        song text, 
        userid int,
        
        PRIMARY KEY (song, userid)
        )"""
try:
    session.execute(create_table3)
except Exception as e:
    print(e)

#### In the next cell we parse the 'event_datafile_new.csv' file row by row and extract the specific columns data and load it into the Third Table "music_library_by_songs"

In [45]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO music_library_by_songs ( 
                firstname, 
                lastname, 
                song, 
                userId) 
        """
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

#### Execute The Third Query

In [46]:
query3 = """SELECT firstname, lastname
            FROM music_library_by_songs
            WHERE song = 'All Hands Against His Own'
"""
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the Three Created tables From the Keyspace

In [48]:
drop_table1 = "DROP TABLE IF EXISTS music_library_by_sessions"
try:
    session.execute(drop_table1)
except Exception as e:
    print(e)
    
    
drop_table2 = "DROP TABLE IF EXISTS music_library_by_users_and_sessions"
try:
    session.execute(drop_table2)
except Exception as e:
    print(e)


drop_table3 = "DROP TABLE IF EXISTS music_library_by_songs"
try:
    session.execute(drop_table3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()